# Codebusters KPIs and Other Usefull Patterns

## import libraries

In [17]:
import pandas as pd
import numpy as np
from pathlib import Path

Import and clean `all_tickets_2022` data set 
- remove unnecessary columns
- rename columns

# Prepare data with all tickets

### Import all service request

Tickets are taken from the following jira filter:<br/> `project = CI AND issuetype in (standardIssueTypes(), "Expense Delivery") AND "Epic Link" is EMPTY AND "Case Number/s" is not EMPTY AND cf[14125] in ("Service Request (SR)") AND resolved is not EMPTY AND resolutiondate >= 2022-12-19

In [18]:
all_sr_origin = pd.read_csv('DataSets/MainTickets/all_service_requests.csv')
all_sr_origin

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Project name,Project type,Project lead,Project description,...,Comment.18,Comment.19,Comment.20,Comment.21,Comment.22,Comment.23,Comment.24,Comment.25,Comment.26,Comment.27
0,Numiga Nordsee - Increase approval timeouts in...,CI-8908,484613,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Munckhof - Change mileage amounts for a location,CI-8889,484189,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Numiga Hensoldt - Change Bank account in SEPA ...,CI-8872,483676,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Numiga Cariad - Change Bank account for SEPA file,CI-8871,483671,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Numiga Elli - Change Bank account in SEPA file,CI-8870,483664,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,Aquila Capital - Expense issue with export and...,CI-6926,393799,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,04/Aug/22 10:36 AM;gmalarski;Export path added...,27/Jan/23 7:54 PM;fraphose;SR is closed since ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,VW - Splitting of approval for expense statements,CI-6911,393585,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403,Porsche - Adjustment of approval process for n...,CI-6901,393357,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
404,Hugo Boss - add 3 new meal receipt types for t...,CI-6825,389774,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Select from the dataset only columns necessary for the further analysis

In [19]:
all_sr = all_sr_origin[['Issue key', 'Issue id', 'Status', 'Reporter', 'Created', 'Due Date', 'Resolved', 'Custom field (Customer/s)','Custom field (Type of Request)']]
all_sr.rename({"Custom field (Customer/s)": "Customer", "Custom field (Type of Request)":"Type of Request"},axis=1, inplace=True)

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_19484\2948666729.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sr.rename({"Custom field (Customer/s)": "Customer", "Custom field (Type of Request)":"Type of Request"},axis=1, inplace=True)


In [20]:
all_sr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Issue key        406 non-null    object
 1   Issue id         406 non-null    int64 
 2   Status           406 non-null    object
 3   Reporter         401 non-null    object
 4   Created          406 non-null    object
 5   Due Date         406 non-null    object
 6   Resolved         406 non-null    object
 7   Customer         406 non-null    object
 8   Type of Request  406 non-null    object
dtypes: int64(1), object(8)
memory usage: 28.7+ KB


In [21]:
all_sr.isnull().sum()

Issue key          0
Issue id           0
Status             0
Reporter           5
Created            0
Due Date           0
Resolved           0
Customer           0
Type of Request    0
dtype: int64

### Convert date table to `datetime` object

In [22]:
all_sr["Created"] = pd.to_datetime(all_sr["Created"])
all_sr["Resolved"] = pd.to_datetime(all_sr["Resolved"])

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_19484\2794271155.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_sr["Created"] = pd.to_datetime(all_sr["Created"])
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_19484\2794271155.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sr["Created"] = pd.to_datetime(all_sr["Created"])
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_19484\2794271155.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_sr["Resolved"

In [23]:
all_sr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Issue key        406 non-null    object        
 1   Issue id         406 non-null    int64         
 2   Status           406 non-null    object        
 3   Reporter         401 non-null    object        
 4   Created          406 non-null    datetime64[ns]
 5   Due Date         406 non-null    object        
 6   Resolved         406 non-null    datetime64[ns]
 7   Customer         406 non-null    object        
 8   Type of Request  406 non-null    object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 28.7+ KB


In [24]:
all_sr.shape

(406, 9)

In [25]:
all_sr["ticket_age"] = all_sr["Resolved"] - all_sr["Created"]
all_sr

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_19484\2540177896.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sr["ticket_age"] = all_sr["Resolved"] - all_sr["Created"]


,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age
0,CI-8908,484613,Closed,alejandro.ruizrobles,2023-06-29 16:16:00,06/Jul/23 12:00 AM,2023-07-03 14:27:00,Numiga,Service Request (SR),3 days 22:11:00
1,CI-8889,484189,Closed,alejandro.ruizrobles,2023-06-28 11:21:00,06/Jul/23 12:00 AM,2023-07-04 15:41:00,Nedschroef,Service Request (SR),6 days 04:20:00
2,CI-8872,483676,Closed,alejandro.ruizrobles,2023-06-26 13:21:00,03/Jul/23 12:00 AM,2023-07-03 14:21:00,Numiga,Service Request (SR),7 days 01:00:00
3,CI-8871,483671,Closed,alejandro.ruizrobles,2023-06-26 13:14:00,03/Jul/23 12:00 AM,2023-07-03 14:22:00,Numiga,Service Request (SR),7 days 01:08:00
4,CI-8870,483664,Closed,alejandro.ruizrobles,2023-06-26 13:05:00,03/Jul/23 12:00 AM,2023-07-03 14:25:00,Numiga,Service Request (SR),7 days 01:20:00
...,...,...,...,...,...,...,...,...,...,...
401,CI-6926,393799,Closed,NaN,2022-05-25 12:42:00,12/Jul/22 12:00 AM,2023-01-27 19:53:00,Aquila Capital,Service Request (SR),247 days 07:11:00
402,CI-6911,393585,Closed,NaN,2022-05-24 15:11:00,11/Aug/22 12:00 AM,2023-01-24 10:33:00,VW,Service Request (SR),244 days 19:22:00
403,CI-6901,393357,Closed,NaN,2022-05-23 13:21:00,30/May/22 12:00 AM,2023-03-21 13:06:00,Porsche,Service Request (SR),301 days 23:45:00
404,CI-6825,389774,Closed,NaN,2022-05-11 11:16:00,16/May/22 12:00 AM,2023-01-24 12:32:00,Hugo Boss,Service Request (SR),258 days 01:16:00


**Summary**
- there is not SR ticket without `resolved` date
- there is no `null` value in any of the column
- date type columns converted to `datetime` object in order to make calculucations on these columns

# Prepare data with individuals tasks for service requests

## Import and clean `all_individual_tasks` data set 
- we are not able to downaload full data becuase of jira limitation, therefore we need to combine data from differen periods
- remove unnecessary columns
- replace missing `date` values with 0
- drop rows with "null" values

In [26]:
individual_part_1 = pd.read_csv('DataSets/IndividualTasks/from_20220901_till_20221231.csv')
individual_part_2 = pd.read_csv('DataSets/IndividualTasks/from_20221231_till_20230331.csv')
# individual_part_3 = pd.read_csv('DataSets/IndividualTasks/from_20230401_till_20230516.csv')
individual_part_3 = pd.read_csv('DataSets/IndividualTasks/from_20230331_till_20230703.csv')


all_individual_tasks_combined = pd.concat([individual_part_1, individual_part_2, individual_part_3], axis=0)
all_individual_tasks_combined

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_19484\251257024.py:1: DtypeWarning: Columns (216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233) have mixed types. Specify dtype option on import or set low_memory=False.
  individual_part_1 = pd.read_csv('DataSets/IndividualTasks/from_20220901_till_20221231.csv')
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_19484\251257024.py:2: DtypeWarning: Columns (25,222,223,224,225,226,227,228,229,230,264) have mixed types. Specify dtype option on import or set low_memory=False.
  individual_part_2 = pd.read_csv('DataSets/IndividualTasks/from_20221231_till_20230331.csv')
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_19484\251257024.py:4: DtypeWarning: Columns (27,37,38,39,78,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,329,330,331) have mixed types. Specify dtype option on import or set low_memory=False.
  individual_part_3 = pd.read_csv('DataSets/Indivi

,Summary,Issue key,Issue id,Parent id,Issue Type,Status,Project key,Project name,Project type,Project lead,...,Sprint.44,Sprint.45,Sprint.46,Sprint.47,Sprint.48,Comment.2,Comment.3,Comment.4,Comment.5,Comment.6
0,Configure,CI-10230383,438469,437773,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Import to Test System,CI-10230381,438460,437698,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Configure,CI-10230380,438459,437698,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Import in Production,CI-10230377,438438,432387,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Import to Customer Test System,CI-10230376,438437,435618,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3410,Import in Production,CI-10236611,461949,457052,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3411,Communicate,CI-10236609,461942,459898,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3412,Import to Customer Test System,CI-10236608,461941,459898,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3413,Verify,CI-10236607,461938,458212,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
all_individual_tasks_combined[all_individual_tasks_combined["Parent id"] == 456393]

,Summary,Issue key,Issue id,Parent id,Issue Type,Status,Project key,Project name,Project type,Project lead,...,Sprint.44,Sprint.45,Sprint.46,Sprint.47,Sprint.48,Comment.2,Comment.3,Comment.4,Comment.5,Comment.6
7,Import to Test System,CI-10236592,461862,456393,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,Configure,CI-10236584,461851,456393,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401,Review,CI-10235480,456394,456393,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2888,Communicate,CI-10237679,464825,456393,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3327,Import in Production,CI-10236775,462381,456393,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3372,Verify,CI-10236681,462158,456393,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Reset indexex of a combined dataset </br>
Indexes of a new dataset are not appropriate, we need to apply `reset_index` method in order to match them with the number of rows

In [28]:
all_individual_tasks_combined.reset_index(inplace=True)
all_individual_tasks_combined

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_19484\4115877976.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_individual_tasks_combined.reset_index(inplace=True)


,index,Summary,Issue key,Issue id,Parent id,Issue Type,Status,Project key,Project name,Project type,...,Sprint.44,Sprint.45,Sprint.46,Sprint.47,Sprint.48,Comment.2,Comment.3,Comment.4,Comment.5,Comment.6
0,0,Configure,CI-10230383,438469,437773,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Import to Test System,CI-10230381,438460,437698,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Configure,CI-10230380,438459,437698,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Import in Production,CI-10230377,438438,432387,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Import to Customer Test System,CI-10230376,438437,435618,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9984,3410,Import in Production,CI-10236611,461949,457052,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9985,3411,Communicate,CI-10236609,461942,459898,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9986,3412,Import to Customer Test System,CI-10236608,461941,459898,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9987,3413,Verify,CI-10236607,461938,458212,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
all_individual_tasks_combined.to_csv("all_individual_tasks_combined.csv")

### Filter columns and select only the ones needed for further calculations

In [30]:
all_individual_tasks = all_individual_tasks_combined[['Summary','Issue id', 'Parent id', 'Created', 'Resolved']]
all_individual_tasks

,Summary,Issue id,Parent id,Created,Resolved
0,Configure,438469,437773,30/12/2022 13:56,02/01/2023 13:18
1,Import to Test System,438460,437698,30/12/2022 13:14,02/01/2023 13:33
2,Configure,438459,437698,30/12/2022 13:14,30/12/2022 13:14
3,Import in Production,438438,432387,30/12/2022 11:38,05/01/2023 09:48
4,Import to Customer Test System,438437,435618,30/12/2022 11:01,03/01/2023 12:17
...,...,...,...,...,...
9984,Import in Production,461949,457052,31/Mar/23 9:08 AM,06/Apr/23 12:39 PM
9985,Communicate,461942,459898,31/Mar/23 9:02 AM,31/Mar/23 9:02 AM
9986,Import to Customer Test System,461941,459898,31/Mar/23 9:02 AM,31/Mar/23 9:02 AM
9987,Verify,461938,458212,31/Mar/23 8:54 AM,31/Mar/23 10:58 AM


In [31]:
all_individual_tasks[all_individual_tasks['Parent id'] == 465210]

,Summary,Issue id,Parent id,Created,Resolved
7612,Communicate,477694,465210,07/Jun/23 1:56 PM,NaN
7679,Import in Production,477344,465210,06/Jun/23 3:45 PM,07/Jun/23 1:56 PM
7680,Configure,477343,465210,06/Jun/23 3:45 PM,06/Jun/23 3:45 PM
7681,Review,477342,465210,06/Jun/23 3:44 PM,06/Jun/23 3:45 PM
9160,Explain,466478,465210,20/Apr/23 8:22 AM,26/Apr/23 10:33 AM
9367,Review,465211,465210,17/Apr/23 9:43 AM,20/Apr/23 8:22 AM


### Convert date like columns to `datetime` object

In [32]:
all_individual_tasks["Created"] = pd.to_datetime(all_individual_tasks["Created"], dayfirst=True, format='mixed')
all_individual_tasks["Resolved"] = pd.to_datetime(all_individual_tasks["Resolved"], dayfirst=True,  format='mixed')
all_individual_tasks.info()

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_19484\2151095576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks["Created"] = pd.to_datetime(all_individual_tasks["Created"], dayfirst=True, format='mixed')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9989 entries, 0 to 9988
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Summary    9989 non-null   object        
 1   Issue id   9989 non-null   int64         
 2   Parent id  9989 non-null   int64         
 3   Created    9989 non-null   datetime64[ns]
 4   Resolved   9662 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 390.3+ KB


C:\Users\gmalarski\AppData\Local\Temp\ipykernel_19484\2151095576.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks["Resolved"] = pd.to_datetime(all_individual_tasks["Resolved"], dayfirst=True,  format='mixed')


In [33]:
all_individual_tasks['Summary'].value_counts()

Summary
Configure                               1765
Review                                  1717
Verify                                  1637
Communicate                             1437
Import to Test System                   1291
Import in Production                    1126
Import to Customer Test System           610
Explain                                  229
Clarify                                   92
Do                                        80
CLONE - Review                             1
Contact Mattia about risks assesment       1
Rename the repository                      1
Check secrets with ITS                     1
Configure2                                 1
Name: count, dtype: int64

### Check if there are any `null` values

In [34]:
all_individual_tasks.isnull().sum()

Summary        0
Issue id       0
Parent id      0
Created        0
Resolved     327
dtype: int64

### Drop all rows with null value

In [35]:
all_individual_tasks_without_null = all_individual_tasks.dropna()
all_individual_tasks_without_null.isnull().sum()

Summary      0
Issue id     0
Parent id    0
Created      0
Resolved     0
dtype: int64

### Identify and drop duplicated rows

In [36]:
all_individual_tasks_duplicated_rows = all_individual_tasks_without_null[all_individual_tasks_without_null.duplicated()]
all_individual_tasks_duplicated_rows

,Summary,Issue id,Parent id,Created,Resolved


In [37]:
all_individual_tasks_no_duplicates = all_individual_tasks_without_null.drop_duplicates()
all_individual_tasks_no_duplicates

,Summary,Issue id,Parent id,Created,Resolved
0,Configure,438469,437773,2022-12-30 13:56:00,2023-01-02 13:18:00
1,Import to Test System,438460,437698,2022-12-30 13:14:00,2023-01-02 13:33:00
2,Configure,438459,437698,2022-12-30 13:14:00,2022-12-30 13:14:00
3,Import in Production,438438,432387,2022-12-30 11:38:00,2023-01-05 09:48:00
4,Import to Customer Test System,438437,435618,2022-12-30 11:01:00,2023-01-03 12:17:00
...,...,...,...,...,...
9984,Import in Production,461949,457052,2023-03-31 09:08:00,2023-04-06 12:39:00
9985,Communicate,461942,459898,2023-03-31 09:02:00,2023-03-31 09:02:00
9986,Import to Customer Test System,461941,459898,2023-03-31 09:02:00,2023-03-31 09:02:00
9987,Verify,461938,458212,2023-03-31 08:54:00,2023-03-31 10:58:00


## Summary

- We are interested only in completed tickets so we have droped rows which contain `null` in `resolved` columns
- Data is filtered and cleaned we are ready for analysis part

# Data Analysis 

# Daterminate how long each ticket was in particular individual task

- calculate time for eacch subtask
- calculate how long each ticket was in review
- calculate how long each ticket was "on codebusters"
- calculate how long each ticket was "outside codebusters"


## Calculate time spent in each of the subtask

In [38]:
all_individual_tasks_time_calculation = all_individual_tasks_no_duplicates
all_individual_tasks_time_calculation["time_in"] = all_individual_tasks_time_calculation["Resolved"] - all_individual_tasks_time_calculation["Created"]

### List all available statuses of the CI ticket subtask

In [39]:
all_individual_tasks_time_calculation["Summary"].value_counts()

Summary
Configure                         1735
Review                            1608
Verify                            1597
Communicate                       1360
Import to Test System             1284
Import in Production              1117
Import to Customer Test System     596
Explain                            217
Clarify                             89
Do                                  58
Configure2                           1
Name: count, dtype: int64

In [40]:
all_individual_tasks_time_calculation.shape

(9662, 6)

In [95]:
all_individual_tasks_time_calculation[all_individual_tasks_time_calculation["Parent id"] == 438429]

,Summary,Issue id,Parent id,Created,Resolved,time_in
11,Review,438430,438429,2022-12-30 10:21:00,2023-01-09 17:06:00,10 days 06:45:00
6048,Communicate,442401,438429,2023-01-18 15:14:00,2023-02-17 08:26:00,29 days 17:12:00
6060,Import in Production,442287,438429,2023-01-18 10:51:00,2023-01-18 15:14:00,0 days 04:23:00
6313,Verify,440581,438429,2023-01-10 17:43:00,2023-01-17 17:13:00,6 days 23:30:00
6315,Import to Test System,440473,438429,2023-01-10 16:32:00,2023-01-10 17:43:00,0 days 01:11:00
6351,Configure,440062,438429,2023-01-09 17:06:00,2023-01-10 16:32:00,0 days 23:26:00


In [42]:
all_individual_tasks_time_calculation[all_individual_tasks_time_calculation["Parent id"] == 456393]["time_in"].sum()

Timedelta('28 days 23:19:00')

In [43]:
all_individual_tasks_reivew = all_individual_tasks_time_calculation[all_individual_tasks_time_calculation['Summary'] == 'Review']
all_individual_tasks_reivew.rename({"time_in": "time_in_review"},axis=1, inplace=True)
all_individual_tasks_reivew.reset_index(inplace=True)
all_individual_tasks_reivew

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_19484\3555403275.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks_reivew.rename({"time_in": "time_in_review"},axis=1, inplace=True)


,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review
0,11,Review,438430,438429,2022-12-30 10:21:00,2023-01-09 17:06:00,10 days 06:45:00
1,19,Review,438405,436853,2022-12-29 16:55:00,2023-01-04 09:50:00,5 days 16:55:00
2,25,Review,438383,438382,2022-12-29 14:47:00,2023-01-04 11:44:00,5 days 20:57:00
3,38,Review,438363,438360,2022-12-29 11:45:00,2022-12-29 12:21:00,0 days 00:36:00
4,40,Review,438361,438360,2022-12-29 11:30:00,2022-12-29 11:42:00,0 days 00:12:00
...,...,...,...,...,...,...,...
1603,9953,Review,462120,462119,2023-03-31 13:37:00,2023-04-18 16:44:00,18 days 03:07:00
1604,9954,Review,462118,462117,2023-03-31 13:35:00,2023-04-18 16:44:00,18 days 03:09:00
1605,9955,Review,462116,462115,2023-03-31 13:32:00,2023-04-18 10:34:00,17 days 21:02:00
1606,9974,Review,462016,462015,2023-03-31 10:19:00,2023-04-05 12:30:00,5 days 02:11:00


In [44]:
all_sr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Issue key        406 non-null    object         
 1   Issue id         406 non-null    int64          
 2   Status           406 non-null    object         
 3   Reporter         401 non-null    object         
 4   Created          406 non-null    datetime64[ns] 
 5   Due Date         406 non-null    object         
 6   Resolved         406 non-null    datetime64[ns] 
 7   Customer         406 non-null    object         
 8   Type of Request  406 non-null    object         
 9   ticket_age       406 non-null    timedelta64[ns]
dtypes: datetime64[ns](2), int64(1), object(6), timedelta64[ns](1)
memory usage: 31.8+ KB


### Grup tickets by the `Parent id` and sum total time spent in `review` subtask


In [45]:
all_individual_tasks_reivew[all_individual_tasks_reivew["Parent id"] == 465210]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review
1253,7681,Review,477342,465210,2023-06-06 15:44:00,2023-06-06 15:45:00,0 days 00:01:00
1513,9367,Review,465211,465210,2023-04-17 09:43:00,2023-04-20 08:22:00,2 days 22:39:00


In [46]:
all_individual_tasks_reivew_sum = all_individual_tasks_reivew.groupby(by=["Parent id"], dropna=False)["time_in_review"].sum()
all_individual_tasks_reivew_sum = all_individual_tasks_reivew_sum.to_frame()
all_individual_tasks_reivew_sum.reset_index(inplace=True)
all_individual_tasks_reivew_sum

,Parent id,time_in_review
0,336670,0 days 00:01:00
1,349721,0 days 00:02:00
2,393585,3 days 13:22:00
3,399228,52 days 06:54:00
4,402477,0 days 18:56:00
...,...,...
1431,485193,0 days 22:15:00
1432,485356,0 days 04:19:00
1433,485367,0 days 00:05:00
1434,485418,0 days 01:44:00


In [47]:
all_individual_tasks_reivew_sum[all_individual_tasks_reivew_sum["Parent id"] == 465210]

,Parent id,time_in_review
1082,465210,2 days 22:40:00


In [48]:
all_individual_tasks_reivew_sum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1436 entries, 0 to 1435
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype          
---  ------          --------------  -----          
 0   Parent id       1436 non-null   int64          
 1   time_in_review  1436 non-null   timedelta64[ns]
dtypes: int64(1), timedelta64[ns](1)
memory usage: 22.6 KB


In [49]:
all_individual_tasks_reivew_sum[all_individual_tasks_reivew_sum["Parent id"] == 465210]

,Parent id,time_in_review
1082,465210,2 days 22:40:00


### Check on the particular ticket if the total time is summed correctly

In [50]:
all_individual_tasks_reivew.groupby("Parent id").count()

,index,Summary,Issue id,Created,Resolved,time_in_review
Parent id,,,,,,
336670,1,1,1,1,1,1
349721,1,1,1,1,1,1
393585,1,1,1,1,1,1
399228,3,3,3,3,3,3
402477,2,2,2,2,2,2
...,...,...,...,...,...,...
485193,1,1,1,1,1,1
485356,1,1,1,1,1,1
485367,2,2,2,2,2,2


In [51]:
all_individual_tasks_reivew.loc[all_individual_tasks_reivew["Parent id"] == 345462]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review


In [52]:
type(all_individual_tasks_reivew_sum)

pandas.core.frame.DataFrame

In [53]:
all_individual_tasks_reivew_sum_df = all_individual_tasks_reivew_sum
all_individual_tasks_reivew_sum_df.reset_index(inplace=True)
all_individual_tasks_reivew_sum_df

,index,Parent id,time_in_review
0,0,336670,0 days 00:01:00
1,1,349721,0 days 00:02:00
2,2,393585,3 days 13:22:00
3,3,399228,52 days 06:54:00
4,4,402477,0 days 18:56:00
...,...,...,...
1431,1431,485193,0 days 22:15:00
1432,1432,485356,0 days 04:19:00
1433,1433,485367,0 days 00:05:00
1434,1434,485418,0 days 01:44:00


In [54]:
all_individual_tasks_reivew_sum_df.loc[all_individual_tasks_reivew_sum_df["Parent id"] == 345462]

,index,Parent id,time_in_review


In [55]:
all_individual_tasks_reivew_sum_df.isnull().sum()

index             0
Parent id         0
time_in_review    0
dtype: int64

In [56]:
all_individual_tasks_reivew_sum_df

,index,Parent id,time_in_review
0,0,336670,0 days 00:01:00
1,1,349721,0 days 00:02:00
2,2,393585,3 days 13:22:00
3,3,399228,52 days 06:54:00
4,4,402477,0 days 18:56:00
...,...,...,...
1431,1431,485193,0 days 22:15:00
1432,1432,485356,0 days 04:19:00
1433,1433,485367,0 days 00:05:00
1434,1434,485418,0 days 01:44:00


**Summary:**
- There are some tickets which have been more than once in the `Review` step
- We can notice that after gruping and summing tickets, case with 'Parent id' `345462` has total time equals more than `8 days`, which is the sum of the time spend in previous two `Review` subtasks

### Merge `all_sr` dataset with `all_individual_tasks_reivew_sum_df`

In [57]:
all_sr_review_time = pd.merge(all_sr, all_individual_tasks_reivew_sum_df[['Parent id', 'time_in_review']], how="left",left_on = "Issue id",right_on = "Parent id")
all_sr_review_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_in_review
0,CI-8908,484613,Closed,alejandro.ruizrobles,2023-06-29 16:16:00,06/Jul/23 12:00 AM,2023-07-03 14:27:00,Numiga,Service Request (SR),3 days 22:11:00,484613.0,0 days 17:35:00
1,CI-8889,484189,Closed,alejandro.ruizrobles,2023-06-28 11:21:00,06/Jul/23 12:00 AM,2023-07-04 15:41:00,Nedschroef,Service Request (SR),6 days 04:20:00,484189.0,5 days 00:12:00
2,CI-8872,483676,Closed,alejandro.ruizrobles,2023-06-26 13:21:00,03/Jul/23 12:00 AM,2023-07-03 14:21:00,Numiga,Service Request (SR),7 days 01:00:00,483676.0,2 days 02:44:00
3,CI-8871,483671,Closed,alejandro.ruizrobles,2023-06-26 13:14:00,03/Jul/23 12:00 AM,2023-07-03 14:22:00,Numiga,Service Request (SR),7 days 01:08:00,483671.0,2 days 02:51:00
4,CI-8870,483664,Closed,alejandro.ruizrobles,2023-06-26 13:05:00,03/Jul/23 12:00 AM,2023-07-03 14:25:00,Numiga,Service Request (SR),7 days 01:20:00,483664.0,2 days 02:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
401,CI-6926,393799,Closed,NaN,2022-05-25 12:42:00,12/Jul/22 12:00 AM,2023-01-27 19:53:00,Aquila Capital,Service Request (SR),247 days 07:11:00,NaN,NaT
402,CI-6911,393585,Closed,NaN,2022-05-24 15:11:00,11/Aug/22 12:00 AM,2023-01-24 10:33:00,VW,Service Request (SR),244 days 19:22:00,393585.0,3 days 13:22:00
403,CI-6901,393357,Closed,NaN,2022-05-23 13:21:00,30/May/22 12:00 AM,2023-03-21 13:06:00,Porsche,Service Request (SR),301 days 23:45:00,NaN,NaT
404,CI-6825,389774,Closed,NaN,2022-05-11 11:16:00,16/May/22 12:00 AM,2023-01-24 12:32:00,Hugo Boss,Service Request (SR),258 days 01:16:00,NaN,NaT


In [58]:
all_sr_review_time.isnull().sum()

Issue key           0
Issue id            0
Status              0
Reporter            5
Created             0
Due Date            0
Resolved            0
Customer            0
Type of Request     0
ticket_age          0
Parent id          13
time_in_review     13
dtype: int64

### Investigate `null` rows after merging  main SR tickets with individual subtasks

In [59]:
null_rows = all_sr_review_time[all_sr_review_time["time_in_review"].isnull()]
null_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, 28 to 404
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Issue key        13 non-null     object         
 1   Issue id         13 non-null     int64          
 2   Status           13 non-null     object         
 3   Reporter         9 non-null      object         
 4   Created          13 non-null     datetime64[ns] 
 5   Due Date         13 non-null     object         
 6   Resolved         13 non-null     datetime64[ns] 
 7   Customer         13 non-null     object         
 8   Type of Request  13 non-null     object         
 9   ticket_age       13 non-null     timedelta64[ns]
 10  Parent id        0 non-null      float64        
 11  time_in_review   0 non-null      timedelta64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(6), timedelta64[ns](2)
memory usage: 1.3+ KB


In [60]:
null_rows

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_in_review
28,CI-8680,474435,Closed,alejandro.ruizrobles,2023-05-25 10:38:00,02/Jun/23 12:00 AM,2023-06-14 08:36:00,LKW Walter,Service Request (SR),19 days 21:58:00,NaN,NaT
389,CI-7306,409890,Closed,ssanz,2022-08-23 15:29:00,30/Aug/22 12:00 AM,2022-12-22 14:49:00,Porsche,Service Request (SR),120 days 23:20:00,NaN,NaT
393,CI-7226,407535,Closed,NaN,2022-08-09 15:05:00,16/Aug/22 12:00 AM,2023-01-18 06:29:00,Porsche,Service Request (SR),161 days 15:24:00,NaN,NaT
394,CI-7221,407289,Closed,askarja,2022-08-08 23:50:00,15/Aug/22 12:00 AM,2023-01-24 10:31:00,Audi,Service Request (SR),168 days 10:41:00,NaN,NaT
395,CI-7205,406532,Closed,nsanchezruiz,2022-08-04 11:54:00,02/May/23 12:00 AM,2023-04-10 08:47:00,Hugo Boss,Service Request (SR),248 days 20:53:00,NaN,NaT
396,CI-7179,405159,Closed,askarja,2022-07-27 19:06:00,04/Aug/22 12:00 AM,2022-12-29 10:43:00,Eberspacher,Service Request (SR),154 days 15:37:00,NaN,NaT
397,CI-7178,405141,Closed,askarja,2022-07-27 16:18:00,03/Aug/22 12:00 AM,2023-01-24 10:32:00,msg systems,Service Request (SR),180 days 18:14:00,NaN,NaT
398,CI-7167,404691,Closed,askarja,2022-07-25 13:03:00,01/Aug/22 12:00 AM,2023-01-24 10:34:00,Rheinmetall,Service Request (SR),182 days 21:31:00,NaN,NaT
399,CI-7156,403721,Closed,askarja,2022-07-19 18:23:00,26/Jul/22 12:00 AM,2023-01-27 20:00:00,Traton SE,Service Request (SR),192 days 01:37:00,NaN,NaT
400,CI-7029,398756,Closed,askarja,2022-06-23 14:32:00,30/Jun/22 12:00 AM,2023-01-27 19:55:00,TeamBank AG,Service Request (SR),218 days 05:23:00,NaN,NaT


In [61]:
all_sr_review_time_without_null = all_sr_review_time.dropna()
all_sr_review_time_without_null.isnull().sum()

Issue key          0
Issue id           0
Status             0
Reporter           0
Created            0
Due Date           0
Resolved           0
Customer           0
Type of Request    0
ticket_age         0
Parent id          0
time_in_review     0
dtype: int64

**Summary**:
- Ror tickets like `CI-8488`revew time equals 0, so we need to drop it, because it can leads to wrong KPIs calculations

# TO DO
- update the lates dateset

## Determinate time spent on codebusters

- calculate the total time ticket was in subtasks "Configure", "Review", "mport to Test System", "Import in Production", "Import to Customer Test System"
- apply the results to all tickets dataset

In [62]:
all_individual_with_codebusters_time = all_individual_tasks_time_calculation.loc[all_individual_tasks_time_calculation["Summary"].isin(["Configure", "Review", "Import to Test System", "Import in Production", "Import to Customer Test System", "Import to Test", "Import to Staging System"])]

Summarize time for each codebusters subtask 

In [63]:
all_individual_with_codebusters_time.groupby("Parent id").count()

,Summary,Issue id,Created,Resolved,time_in
Parent id,,,,,
336670,1,1,1,1,1
349721,3,3,3,3,3
356993,3,3,3,3,3
359781,3,3,3,3,3
360791,1,1,1,1,1
...,...,...,...,...,...
485193,3,3,3,3,3
485356,2,2,2,2,2
485367,6,6,6,6,6


In [64]:
all_individual_with_codebusters_time[all_individual_with_codebusters_time['Parent id'] == 329036]

,Summary,Issue id,Parent id,Created,Resolved,time_in


In [65]:
all_individual_with_codebusters_time_sum = all_individual_with_codebusters_time.groupby("Parent id")["time_in"].sum()
all_individual_with_codebusters_time_sum_df = all_individual_with_codebusters_time_sum.to_frame()
all_individual_with_codebusters_time_sum_df.rename({"time_in": "time_on_codebusters"},axis=1, inplace=True)
all_individual_with_codebusters_time_sum_df.reset_index(inplace=True)
all_individual_with_codebusters_time_sum_df

,Parent id,time_on_codebusters
0,336670,0 days 00:01:00
1,349721,65 days 00:18:00
2,356993,18 days 04:09:00
3,359781,29 days 18:25:00
4,360791,0 days 00:01:00
...,...,...
1551,485193,0 days 22:22:00
1552,485356,0 days 04:20:00
1553,485367,0 days 00:06:00
1554,485418,0 days 01:44:00


In [66]:
# all_individual_tasks_reivew_sum_df = all_individual_tasks_reivew_sum.to_frame()
# all_individual_tasks_reivew_sum_df.reset_index(inplace=True)
# all_individual_tasks_reivew_sum_df

### Merge total time on codebusters with all jira ticekts dataset

In [67]:
all_sr_codebusters_time = pd.merge(all_sr, all_individual_with_codebusters_time_sum_df, how="inner",left_on = "Issue id",right_on = "Parent id")
all_sr_codebusters_time 

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_on_codebusters
0,CI-8908,484613,Closed,alejandro.ruizrobles,2023-06-29 16:16:00,06/Jul/23 12:00 AM,2023-07-03 14:27:00,Numiga,Service Request (SR),3 days 22:11:00,484613,3 days 19:12:00
1,CI-8889,484189,Closed,alejandro.ruizrobles,2023-06-28 11:21:00,06/Jul/23 12:00 AM,2023-07-04 15:41:00,Nedschroef,Service Request (SR),6 days 04:20:00,484189,6 days 02:27:00
2,CI-8872,483676,Closed,alejandro.ruizrobles,2023-06-26 13:21:00,03/Jul/23 12:00 AM,2023-07-03 14:21:00,Numiga,Service Request (SR),7 days 01:00:00,483676,6 days 17:50:00
3,CI-8871,483671,Closed,alejandro.ruizrobles,2023-06-26 13:14:00,03/Jul/23 12:00 AM,2023-07-03 14:22:00,Numiga,Service Request (SR),7 days 01:08:00,483671,6 days 17:57:00
4,CI-8870,483664,Closed,alejandro.ruizrobles,2023-06-26 13:05:00,03/Jul/23 12:00 AM,2023-07-03 14:25:00,Numiga,Service Request (SR),7 days 01:20:00,483664,6 days 18:07:00
...,...,...,...,...,...,...,...,...,...,...,...,...
392,CI-7226,407535,Closed,NaN,2022-08-09 15:05:00,16/Aug/22 12:00 AM,2023-01-18 06:29:00,Porsche,Service Request (SR),161 days 15:24:00,407535,6 days 05:40:00
393,CI-7178,405141,Closed,askarja,2022-07-27 16:18:00,03/Aug/22 12:00 AM,2023-01-24 10:32:00,msg systems,Service Request (SR),180 days 18:14:00,405141,1 days 19:49:00
394,CI-6911,393585,Closed,NaN,2022-05-24 15:11:00,11/Aug/22 12:00 AM,2023-01-24 10:33:00,VW,Service Request (SR),244 days 19:22:00,393585,8 days 18:55:00
395,CI-6901,393357,Closed,NaN,2022-05-23 13:21:00,30/May/22 12:00 AM,2023-03-21 13:06:00,Porsche,Service Request (SR),301 days 23:45:00,393357,0 days 00:00:00


### Investigate `null` rows after merging  main SR tickets with individual subtasks

In [68]:
all_sr_codebusters_time.isnull().sum()

Issue key              0
Issue id               0
Status                 0
Reporter               3
Created                0
Due Date               0
Resolved               0
Customer               0
Type of Request        0
ticket_age             0
Parent id              0
time_on_codebusters    0
dtype: int64

In [69]:
all_sr_codebusters_time_without_null = all_sr_codebusters_time.dropna()
all_sr_codebusters_time_without_null.isnull().sum()

Issue key              0
Issue id               0
Status                 0
Reporter               0
Created                0
Due Date               0
Resolved               0
Customer               0
Type of Request        0
ticket_age             0
Parent id              0
time_on_codebusters    0
dtype: int64

In [70]:
all_sr_codebusters_time_without_null

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_on_codebusters
0,CI-8908,484613,Closed,alejandro.ruizrobles,2023-06-29 16:16:00,06/Jul/23 12:00 AM,2023-07-03 14:27:00,Numiga,Service Request (SR),3 days 22:11:00,484613,3 days 19:12:00
1,CI-8889,484189,Closed,alejandro.ruizrobles,2023-06-28 11:21:00,06/Jul/23 12:00 AM,2023-07-04 15:41:00,Nedschroef,Service Request (SR),6 days 04:20:00,484189,6 days 02:27:00
2,CI-8872,483676,Closed,alejandro.ruizrobles,2023-06-26 13:21:00,03/Jul/23 12:00 AM,2023-07-03 14:21:00,Numiga,Service Request (SR),7 days 01:00:00,483676,6 days 17:50:00
3,CI-8871,483671,Closed,alejandro.ruizrobles,2023-06-26 13:14:00,03/Jul/23 12:00 AM,2023-07-03 14:22:00,Numiga,Service Request (SR),7 days 01:08:00,483671,6 days 17:57:00
4,CI-8870,483664,Closed,alejandro.ruizrobles,2023-06-26 13:05:00,03/Jul/23 12:00 AM,2023-07-03 14:25:00,Numiga,Service Request (SR),7 days 01:20:00,483664,6 days 18:07:00
...,...,...,...,...,...,...,...,...,...,...,...,...
389,CI-7235,408302,Closed,askarja,2022-08-13 13:35:00,06/Sep/22 12:00 AM,2023-01-24 10:30:00,Rheinmetall,Service Request (SR),163 days 20:55:00,408302,6 days 20:20:00
390,CI-7229,407702,Closed,ssanz,2022-08-10 12:43:00,17/Aug/22 12:00 AM,2023-01-17 15:55:00,Vaillant,Service Request (SR),160 days 03:12:00,407702,42 days 11:31:00
391,CI-7227,407574,Closed,askarja,2022-08-10 00:39:00,30/Sep/22 12:00 AM,2023-01-24 11:52:00,Rheinmetall,Service Request (SR),167 days 11:13:00,407574,102 days 23:41:00
393,CI-7178,405141,Closed,askarja,2022-07-27 16:18:00,03/Aug/22 12:00 AM,2023-01-24 10:32:00,msg systems,Service Request (SR),180 days 18:14:00,405141,1 days 19:49:00


**Summary:**

## Determinate time spent outside codebusters

- calculate the total time ticket was in one of the subtask "Explain", "Verify", "Clarify", "Communicate"
- apply results to all SR tickets dataset

In [71]:
all_individual_others_time = all_individual_tasks_time_calculation.loc[all_individual_tasks_time_calculation["Summary"].isin(["Explain", "Verify", "Clarify", "Communicate"])]
all_individual_others_time 

,Summary,Issue id,Parent id,Created,Resolved,time_in
7,Communicate,438434,427628,2022-12-30 10:49:00,2023-01-02 11:04:00,3 days 00:15:00
12,Verify,438428,421362,2022-12-30 09:59:00,2022-12-30 10:52:00,0 days 00:53:00
13,Verify,438427,435618,2022-12-30 09:58:00,2022-12-30 11:01:00,0 days 01:03:00
15,Verify,438421,436629,2022-12-30 09:13:00,2023-01-04 12:56:00,5 days 03:43:00
16,Verify,438420,434536,2022-12-30 09:05:00,2023-01-05 14:53:00,6 days 05:48:00
...,...,...,...,...,...,...
9972,Verify,462018,461596,2023-03-31 10:23:00,2023-03-31 10:58:00,0 days 00:35:00
9973,Verify,462017,461612,2023-03-31 10:23:00,2023-03-31 10:59:00,0 days 00:36:00
9979,Communicate,462005,457190,2023-03-31 10:04:00,2023-04-24 18:12:00,24 days 08:08:00
9985,Communicate,461942,459898,2023-03-31 09:02:00,2023-03-31 09:02:00,0 days 00:00:00


In [72]:
all_individual_others_time[all_individual_others_time["Parent id"] == 456393]

,Summary,Issue id,Parent id,Created,Resolved,time_in
9462,Communicate,464825,456393,2023-04-13 14:21:00,2023-04-13 14:22:00,0 days 00:01:00
9946,Verify,462158,456393,2023-03-31 14:42:00,2023-04-03 11:08:00,2 days 20:26:00


Summarize time for each subtask outside codeubsters

In [73]:
all_individual_others_time_sum = all_individual_others_time.groupby("Parent id")["time_in"].sum()
all_individual_others_time_sum_df = all_individual_others_time_sum.to_frame()
all_individual_others_time_sum_df.rename({"time_in": "time_outside_codebusters"},axis=1, inplace=True)
all_individual_others_time_sum_df.reset_index(inplace=True)
all_individual_others_time_sum_df

,Parent id,time_outside_codebusters
0,308144,0 days 00:01:00
1,331914,0 days 00:00:00
2,335272,0 days 00:00:00
3,336670,3 days 15:01:00
4,338034,17 days 05:13:00
...,...,...
1501,484570,0 days 00:00:00
1502,484613,0 days 02:59:00
1503,484946,0 days 00:00:00
1504,485057,0 days 22:33:00


In [74]:
all_individual_others_time_sum_df[all_individual_others_time_sum_df["Parent id"] == 456393]

,Parent id,time_outside_codebusters
1068,456393,2 days 20:27:00


In [75]:
# all_individual_with_codebusters_time_sum = all_individual_with_codebusters_time.groupby("Parent id")["time_in"].sum()
# all_individual_with_codebusters_time_sum_df = all_individual_with_codebusters_time_sum.to_frame()
# all_individual_with_codebusters_time_sum_df.rename({"time_in": "time_on_codebusters"},axis=1, inplace=True)
# all_individual_with_codebusters_time_sum_df.reset_index(inplace=True)
# all_individual_with_codebusters_time_sum_df

### Merge `all_sr` with `all_individual_others_time_sum_df` dataset

In [76]:
all_sr_outside_codebusters_time = pd.merge(all_sr, all_individual_others_time_sum_df, how="inner",left_on = "Issue id",right_on = "Parent id")
all_sr_outside_codebusters_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_outside_codebusters
0,CI-8908,484613,Closed,alejandro.ruizrobles,2023-06-29 16:16:00,06/Jul/23 12:00 AM,2023-07-03 14:27:00,Numiga,Service Request (SR),3 days 22:11:00,484613,0 days 02:59:00
1,CI-8889,484189,Closed,alejandro.ruizrobles,2023-06-28 11:21:00,06/Jul/23 12:00 AM,2023-07-04 15:41:00,Nedschroef,Service Request (SR),6 days 04:20:00,484189,0 days 01:53:00
2,CI-8872,483676,Closed,alejandro.ruizrobles,2023-06-26 13:21:00,03/Jul/23 12:00 AM,2023-07-03 14:21:00,Numiga,Service Request (SR),7 days 01:00:00,483676,0 days 07:10:00
3,CI-8871,483671,Closed,alejandro.ruizrobles,2023-06-26 13:14:00,03/Jul/23 12:00 AM,2023-07-03 14:22:00,Numiga,Service Request (SR),7 days 01:08:00,483671,0 days 07:11:00
4,CI-8870,483664,Closed,alejandro.ruizrobles,2023-06-26 13:05:00,03/Jul/23 12:00 AM,2023-07-03 14:25:00,Numiga,Service Request (SR),7 days 01:20:00,483664,0 days 07:13:00
...,...,...,...,...,...,...,...,...,...,...,...,...
395,CI-7179,405159,Closed,askarja,2022-07-27 19:06:00,04/Aug/22 12:00 AM,2022-12-29 10:43:00,Eberspacher,Service Request (SR),154 days 15:37:00,405159,0 days 00:01:00
396,CI-7178,405141,Closed,askarja,2022-07-27 16:18:00,03/Aug/22 12:00 AM,2023-01-24 10:32:00,msg systems,Service Request (SR),180 days 18:14:00,405141,103 days 14:07:00
397,CI-6911,393585,Closed,NaN,2022-05-24 15:11:00,11/Aug/22 12:00 AM,2023-01-24 10:33:00,VW,Service Request (SR),244 days 19:22:00,393585,117 days 05:54:00
398,CI-6901,393357,Closed,NaN,2022-05-23 13:21:00,30/May/22 12:00 AM,2023-03-21 13:06:00,Porsche,Service Request (SR),301 days 23:45:00,393357,0 days 00:01:00


In [77]:
all_sr_outside_codebusters_time.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    3
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
dtype: int64

**Summary:**

## Combine all created datasets
- In this step we will merge all created erlier datasets: `all_sr_outside_codebusters_time`, `all_sr_codebusters_time` and `all_sr_review_time` 

Merge df containing "review" time with df containg "time on codebusters" 

In [78]:
all_sr_time_on_and_outside_codebusters = pd.merge(all_sr_outside_codebusters_time, all_sr_codebusters_time[['Issue id','time_on_codebusters']],  how="inner",left_on = "Issue id",right_on = "Issue id")
all_sr_time_on_and_outside_codebusters

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters
0,CI-8908,484613,Closed,alejandro.ruizrobles,2023-06-29 16:16:00,06/Jul/23 12:00 AM,2023-07-03 14:27:00,Numiga,Service Request (SR),3 days 22:11:00,484613,0 days 02:59:00,3 days 19:12:00
1,CI-8889,484189,Closed,alejandro.ruizrobles,2023-06-28 11:21:00,06/Jul/23 12:00 AM,2023-07-04 15:41:00,Nedschroef,Service Request (SR),6 days 04:20:00,484189,0 days 01:53:00,6 days 02:27:00
2,CI-8872,483676,Closed,alejandro.ruizrobles,2023-06-26 13:21:00,03/Jul/23 12:00 AM,2023-07-03 14:21:00,Numiga,Service Request (SR),7 days 01:00:00,483676,0 days 07:10:00,6 days 17:50:00
3,CI-8871,483671,Closed,alejandro.ruizrobles,2023-06-26 13:14:00,03/Jul/23 12:00 AM,2023-07-03 14:22:00,Numiga,Service Request (SR),7 days 01:08:00,483671,0 days 07:11:00,6 days 17:57:00
4,CI-8870,483664,Closed,alejandro.ruizrobles,2023-06-26 13:05:00,03/Jul/23 12:00 AM,2023-07-03 14:25:00,Numiga,Service Request (SR),7 days 01:20:00,483664,0 days 07:13:00,6 days 18:07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,CI-7226,407535,Closed,NaN,2022-08-09 15:05:00,16/Aug/22 12:00 AM,2023-01-18 06:29:00,Porsche,Service Request (SR),161 days 15:24:00,407535,126 days 07:06:00,6 days 05:40:00
393,CI-7178,405141,Closed,askarja,2022-07-27 16:18:00,03/Aug/22 12:00 AM,2023-01-24 10:32:00,msg systems,Service Request (SR),180 days 18:14:00,405141,103 days 14:07:00,1 days 19:49:00
394,CI-6911,393585,Closed,NaN,2022-05-24 15:11:00,11/Aug/22 12:00 AM,2023-01-24 10:33:00,VW,Service Request (SR),244 days 19:22:00,393585,117 days 05:54:00,8 days 18:55:00
395,CI-6901,393357,Closed,NaN,2022-05-23 13:21:00,30/May/22 12:00 AM,2023-03-21 13:06:00,Porsche,Service Request (SR),301 days 23:45:00,393357,0 days 00:01:00,0 days 00:00:00


In [79]:
all_sr_time_on_and_outside_codebusters.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    3
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
time_on_codebusters         0
dtype: int64

### Merge `all_tickets_time_on_and_outside_codebusters` with `all_sr_review_time` dataset

In [80]:
all_sr_with_time = pd.merge(all_sr_time_on_and_outside_codebusters, all_sr_review_time[['Issue id','time_in_review']],  how="inner",left_on = "Issue id",right_on = "Issue id")
all_sr_with_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review
0,CI-8908,484613,Closed,alejandro.ruizrobles,2023-06-29 16:16:00,06/Jul/23 12:00 AM,2023-07-03 14:27:00,Numiga,Service Request (SR),3 days 22:11:00,484613,0 days 02:59:00,3 days 19:12:00,0 days 17:35:00
1,CI-8889,484189,Closed,alejandro.ruizrobles,2023-06-28 11:21:00,06/Jul/23 12:00 AM,2023-07-04 15:41:00,Nedschroef,Service Request (SR),6 days 04:20:00,484189,0 days 01:53:00,6 days 02:27:00,5 days 00:12:00
2,CI-8872,483676,Closed,alejandro.ruizrobles,2023-06-26 13:21:00,03/Jul/23 12:00 AM,2023-07-03 14:21:00,Numiga,Service Request (SR),7 days 01:00:00,483676,0 days 07:10:00,6 days 17:50:00,2 days 02:44:00
3,CI-8871,483671,Closed,alejandro.ruizrobles,2023-06-26 13:14:00,03/Jul/23 12:00 AM,2023-07-03 14:22:00,Numiga,Service Request (SR),7 days 01:08:00,483671,0 days 07:11:00,6 days 17:57:00,2 days 02:51:00
4,CI-8870,483664,Closed,alejandro.ruizrobles,2023-06-26 13:05:00,03/Jul/23 12:00 AM,2023-07-03 14:25:00,Numiga,Service Request (SR),7 days 01:20:00,483664,0 days 07:13:00,6 days 18:07:00,2 days 02:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,CI-7226,407535,Closed,NaN,2022-08-09 15:05:00,16/Aug/22 12:00 AM,2023-01-18 06:29:00,Porsche,Service Request (SR),161 days 15:24:00,407535,126 days 07:06:00,6 days 05:40:00,NaT
393,CI-7178,405141,Closed,askarja,2022-07-27 16:18:00,03/Aug/22 12:00 AM,2023-01-24 10:32:00,msg systems,Service Request (SR),180 days 18:14:00,405141,103 days 14:07:00,1 days 19:49:00,NaT
394,CI-6911,393585,Closed,NaN,2022-05-24 15:11:00,11/Aug/22 12:00 AM,2023-01-24 10:33:00,VW,Service Request (SR),244 days 19:22:00,393585,117 days 05:54:00,8 days 18:55:00,3 days 13:22:00
395,CI-6901,393357,Closed,NaN,2022-05-23 13:21:00,30/May/22 12:00 AM,2023-03-21 13:06:00,Porsche,Service Request (SR),301 days 23:45:00,393357,0 days 00:01:00,0 days 00:00:00,NaT


Check for null values

In [81]:
all_sr_with_time.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    3
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
time_on_codebusters         0
time_in_review              4
dtype: int64

In [82]:
all_sr_with_time.shape

(397, 14)

### Drop rows which contain null value

In [83]:
all_sr_with_time_without_null = all_sr_with_time.dropna()
all_sr_with_time_without_null.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    0
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
time_on_codebusters         0
time_in_review              0
dtype: int64

In [84]:
all_sr_with_time_without_null.shape

(392, 14)

In [85]:
# all_sr_with_time_without_null.to_csv('all_sr.csv')

## Convert `time in..` columns to minutes

Convert `timedelta` columns to float

In [86]:
all_sr_with_time_converted = all_sr_with_time_without_null
all_sr_with_time_converted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392 entries, 0 to 396
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype          
---  ------                    --------------  -----          
 0   Issue key                 392 non-null    object         
 1   Issue id                  392 non-null    int64          
 2   Status                    392 non-null    object         
 3   Reporter                  392 non-null    object         
 4   Created                   392 non-null    datetime64[ns] 
 5   Due Date                  392 non-null    object         
 6   Resolved                  392 non-null    datetime64[ns] 
 7   Customer                  392 non-null    object         
 8   Type of Request           392 non-null    object         
 9   ticket_age                392 non-null    timedelta64[ns]
 10  Parent id                 392 non-null    int64          
 11  time_outside_codebusters  392 non-null    timedelta64[ns]
 12  time_on_codeb

Convert `time in..` columns to minutes

In [87]:
all_sr_with_time_converted['time_in_review'] = all_sr_with_time_converted['time_in_review'].dt.total_seconds().div(60)
all_sr_with_time_converted['time_on_codebusters'] = all_sr_with_time_converted['time_on_codebusters'].dt.total_seconds().div(60)
all_sr_with_time_converted['time_outside_codebusters'] = all_sr_with_time_converted['time_outside_codebusters'].dt.total_seconds().div(60)
all_sr_with_time_converted['ticket_age'] = all_sr_with_time_converted['ticket_age'].dt.total_seconds().div(60)

all_sr_with_time_converted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392 entries, 0 to 396
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Issue key                 392 non-null    object        
 1   Issue id                  392 non-null    int64         
 2   Status                    392 non-null    object        
 3   Reporter                  392 non-null    object        
 4   Created                   392 non-null    datetime64[ns]
 5   Due Date                  392 non-null    object        
 6   Resolved                  392 non-null    datetime64[ns]
 7   Customer                  392 non-null    object        
 8   Type of Request           392 non-null    object        
 9   ticket_age                392 non-null    float64       
 10  Parent id                 392 non-null    int64         
 11  time_outside_codebusters  392 non-null    float64       
 12  time_on_codebusters       3

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_19484\497592799.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sr_with_time_converted['time_in_review'] = all_sr_with_time_converted['time_in_review'].dt.total_seconds().div(60)
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_19484\497592799.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sr_with_time_converted['time_on_codebusters'] = all_sr_with_time_converted['time_on_codebusters'].dt.total_seconds().div(60)
C:\Users\gmalarski\AppData\Local

In [88]:
all_sr_with_time_converted.reset_index(inplace=True)
all_sr_with_time_converted

,index,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review
0,0,CI-8908,484613,Closed,alejandro.ruizrobles,2023-06-29 16:16:00,06/Jul/23 12:00 AM,2023-07-03 14:27:00,Numiga,Service Request (SR),5651.0,484613,179.0,5472.0,1055.0
1,1,CI-8889,484189,Closed,alejandro.ruizrobles,2023-06-28 11:21:00,06/Jul/23 12:00 AM,2023-07-04 15:41:00,Nedschroef,Service Request (SR),8900.0,484189,113.0,8787.0,7212.0
2,2,CI-8872,483676,Closed,alejandro.ruizrobles,2023-06-26 13:21:00,03/Jul/23 12:00 AM,2023-07-03 14:21:00,Numiga,Service Request (SR),10140.0,483676,430.0,9710.0,3044.0
3,3,CI-8871,483671,Closed,alejandro.ruizrobles,2023-06-26 13:14:00,03/Jul/23 12:00 AM,2023-07-03 14:22:00,Numiga,Service Request (SR),10148.0,483671,431.0,9717.0,3051.0
4,4,CI-8870,483664,Closed,alejandro.ruizrobles,2023-06-26 13:05:00,03/Jul/23 12:00 AM,2023-07-03 14:25:00,Numiga,Service Request (SR),10160.0,483664,433.0,9727.0,3059.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,387,CI-7365,411330,Closed,askarja,2022-08-30 21:48:00,14/Sep/22 12:00 AM,2023-01-24 10:35:00,Rheinmetall,Service Request (SR),211007.0,411330,180991.0,27845.0,15.0
388,389,CI-7235,408302,Closed,askarja,2022-08-13 13:35:00,06/Sep/22 12:00 AM,2023-01-24 10:30:00,Rheinmetall,Service Request (SR),235975.0,408302,198864.0,9860.0,4.0
389,390,CI-7229,407702,Closed,ssanz,2022-08-10 12:43:00,17/Aug/22 12:00 AM,2023-01-17 15:55:00,Vaillant,Service Request (SR),230592.0,407702,110233.0,61171.0,22420.0
390,391,CI-7227,407574,Closed,askarja,2022-08-10 00:39:00,30/Sep/22 12:00 AM,2023-01-24 11:52:00,Rheinmetall,Service Request (SR),241153.0,407574,1.0,148301.0,148301.0


In [89]:
all_sr_with_time_converted[all_sr_with_time_converted["Issue id"] == 456393]

,index,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review
145,145,CI-8315,456393,Closed,jbonnavion,2023-03-15 15:03:00,22/Mar/23 12:00 AM,2023-04-13 14:22:00,DERPART,Service Request (SR),41719.0,456393,4107.0,37612.0,21685.0


In [90]:
file_path = Path('DataSets/KPIs/all_sr_kpi.csv')
all_sr_with_time_converted.to_csv(path_or_buf=file_path)

# Investigation starts

In [91]:
all_individual_tasks_reivew[all_individual_tasks_reivew["Parent id"] == 438360]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review
3,38,Review,438363,438360,2022-12-29 11:45:00,2022-12-29 12:21:00,0 days 00:36:00
4,40,Review,438361,438360,2022-12-29 11:30:00,2022-12-29 11:42:00,0 days 00:12:00


In [92]:
all_sr[all_sr["Issue id"] == 438360]

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age
288,CI-7938,438360,Closed,nsanchezruiz,2022-12-29 11:30:00,30/Dec/22 12:00 AM,2022-12-29 15:44:00,Ineos,Service Request (SR),0 days 04:14:00


In [93]:
null_rows

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_in_review
28,CI-8680,474435,Closed,alejandro.ruizrobles,2023-05-25 10:38:00,02/Jun/23 12:00 AM,2023-06-14 08:36:00,LKW Walter,Service Request (SR),19 days 21:58:00,NaN,NaT
389,CI-7306,409890,Closed,ssanz,2022-08-23 15:29:00,30/Aug/22 12:00 AM,2022-12-22 14:49:00,Porsche,Service Request (SR),120 days 23:20:00,NaN,NaT
393,CI-7226,407535,Closed,NaN,2022-08-09 15:05:00,16/Aug/22 12:00 AM,2023-01-18 06:29:00,Porsche,Service Request (SR),161 days 15:24:00,NaN,NaT
394,CI-7221,407289,Closed,askarja,2022-08-08 23:50:00,15/Aug/22 12:00 AM,2023-01-24 10:31:00,Audi,Service Request (SR),168 days 10:41:00,NaN,NaT
395,CI-7205,406532,Closed,nsanchezruiz,2022-08-04 11:54:00,02/May/23 12:00 AM,2023-04-10 08:47:00,Hugo Boss,Service Request (SR),248 days 20:53:00,NaN,NaT
396,CI-7179,405159,Closed,askarja,2022-07-27 19:06:00,04/Aug/22 12:00 AM,2022-12-29 10:43:00,Eberspacher,Service Request (SR),154 days 15:37:00,NaN,NaT
397,CI-7178,405141,Closed,askarja,2022-07-27 16:18:00,03/Aug/22 12:00 AM,2023-01-24 10:32:00,msg systems,Service Request (SR),180 days 18:14:00,NaN,NaT
398,CI-7167,404691,Closed,askarja,2022-07-25 13:03:00,01/Aug/22 12:00 AM,2023-01-24 10:34:00,Rheinmetall,Service Request (SR),182 days 21:31:00,NaN,NaT
399,CI-7156,403721,Closed,askarja,2022-07-19 18:23:00,26/Jul/22 12:00 AM,2023-01-27 20:00:00,Traton SE,Service Request (SR),192 days 01:37:00,NaN,NaT
400,CI-7029,398756,Closed,askarja,2022-06-23 14:32:00,30/Jun/22 12:00 AM,2023-01-27 19:55:00,TeamBank AG,Service Request (SR),218 days 05:23:00,NaN,NaT


# Investigation ends

# To Do